# Data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 54608 (delta 15), reused 26 (delta 0), pack-reused 54555
Receiving objects: 100% (54608/54608), 40.71 MiB | 25.88 MiB/s, done.
Resolving deltas: 100% (38167/38167), done.


In [ ]:
!pip install tokenizers==0.8.1RC2

     |████████████████████████████████| 3.0MB 9.8MB/s 


In [ ]:
# !pip install -e ".[dev]"

from tokenizers.models import BPE

In [ ]:
%cd /content/transformers/examples/lxmert

/content/transformers/examples/lxmert


In [ ]:
!pip install wget
!pip install transformers

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=7481ecea3a8f3312f6afa8da31ae02fcb2d1a9fe9f1972a859fc3625694e30f7
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 1.4MB 8.8MB/s 
     |████████████████████████████████| 2.9MB 30.7MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f85b6e79c55edaca80898235a1ab6430175ddd051d17b01b461932e94df9b216
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: tokenizers 0.8.1rc2
    Uninstalling tokenizers-0.8.1rc2:
      Successfully uninstalled tokenizers-0.8.1rc2


In [ ]:
from IPython.display import clear_output, Image, display
import PIL.Image
import io
import json
import torch
import numpy as np
from processing_image import Preprocess
from visualizing_image import SingleImageViz
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils
from transformers import LxmertModel, LxmertTokenizer
import wget
import pickle
import os

In [ ]:
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg)

image_preprocess = Preprocess(frcnn_cfg)
lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")


%s not found in cache or force_download set to True, downloading to %s https://s3.amazonaws.com/models.huggingface.co/bert/unc-nlp/frcnn-vg-finetuned/config.yaml /root/.cache/torch/transformers/tmpey67uksw



loading configuration file cache
%s not found in cache or force_download set to True, downloading to %s https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin /root/.cache/torch/transformers/tmpk0bhaai5



loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /root/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [ ]:
import glob
s_images = '/content/drive/MyDrive/CS685/val2014/'
s_img = glob.glob(s_images + '*.jpg')

In [ ]:
import json
senticap_file = "/content/drive/MyDrive/senticap_dataset/data/senticap_dataset.json"
scap = json.load(open(senticap_file, "r"))

In [ ]:
s_train_img = []
for i in scap['images']:
  if i['split'] == 'train':
    s_train_img.append(s_images+i['filename'])
print(s_train_img[0])

/content/drive/MyDrive/CS685/val2014/COCO_val2014_000000389081.jpg


In [ ]:
s_test_img = []
for i in scap['images']:
  if i['split'] == 'test':
    s_test_img.append(s_images+i['filename'])
print(len(s_test_img))
print(s_test_img[0])

878
/content/drive/MyDrive/CS685/val2014/COCO_val2014_000000497106.jpg


In [ ]:
print(len(s_train_img))
print(len(s_test_img))

1217
878


In [ ]:
model = LxmertModel.from_pretrained('unc-nlp/lxmert-base-uncased')

In [ ]:
from time import time
start = time()
s_train_features = {}
od_train_list = {}
# for idx, img in enumerate(s_train_img[948:1217]): #Melnita
# for idx, img in enumerate(s_train_img[682:948]): #Young
# for idx, img in enumerate(s_train_img[416:682]): #Josh
for idx, img in enumerate(s_train_img[:149]): #Chinmay
  images, sizes, scales_yx = image_preprocess(img)
  output_dict = frcnn(
      images, 
      sizes, 
      scales_yx=scales_yx, 
      padding="max_detections",
      max_detections=frcnn_cfg.max_detections,
      return_tensors="pt"
  )
  normalized_boxes = output_dict.get("normalized_boxes")
  features = output_dict.get("roi_features")
  od_train_list[img] = output_dict
  # inputs = lxmert_tokenizer("What is in the image?", return_tensors="pt")
  # outputs = model(**inputs, visual_feats=features, visual_pos = normalized_boxes)
  # s_train_features[img] = torch.flatten(outputs.vision_output[0])
  print(idx, "Image Done:",img,"seconds: ", time() - start)

0 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000389081.jpg seconds:  16.66849970817566
1 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000290768.jpg seconds:  33.49321126937866
2 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000263323.jpg seconds:  50.8152711391449
3 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000521634.jpg seconds:  66.70559215545654
4 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000226154.jpg seconds:  86.5916736125946
5 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000543492.jpg seconds:  102.89471888542175
6 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000193194.jpg seconds:  120.71813201904297
7 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000212573.jpg seconds:  140.7256622314453
8 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000545978.jpg seconds:  159.25845575332642
9 Image Done: /con

In [ ]:
# with open("/content/drive/MyDrive/stfeatures_149-415.pkl", 'rb') as f:
#   s_train_features = pickle.load( f)
# len(s_train_features)

267

In [ ]:
with open("/content/drive/MyDrive/ICDl/odtl_0-149.pkl", 'wb') as f:
  pickle.dump(od_train_list, f)

In [ ]:
s_train_features['/content/drive/MyDrive/CS685/val2014/COCO_val2014_000000005107.jpg'].shape

torch.Size([27648])

In [ ]:
%mv /content/drive/MyDrive/odtl_149-415.pkl /content/drive/MyDrive/ICDl

In [ ]:
start = time()
# s_test_features = {}
# od_test_list = {}
# for idx, img in enumerate(s_test_img[657:878]): #Melnita
# for idx, img in enumerate(s_test_img[438:657]): #Young
# for idx, img in enumerate(s_test_img[219:438]): #Josh
for idx, img in enumerate(s_test_img[13:219]): #Chinmay
  images, sizes, scales_yx = image_preprocess(img)
  output_dict = frcnn(
      images, 
      sizes, 
      scales_yx=scales_yx, 
      padding="max_detections",
      max_detections=frcnn_cfg.max_detections,
      return_tensors="pt"
  )
  normalized_boxes = output_dict.get("normalized_boxes")
  features = output_dict.get("roi_features")
  od_test_list[img] = output_dict
  inputs = lxmert_tokenizer("What is in the image?", return_tensors="pt")
  outputs = model(**inputs, visual_feats=features, visual_pos = normalized_boxes)
  s_test_features[img] = torch.flatten(outputs.vision_output[0])
  print(idx, "Image Done:",img,"seconds: ", time() - start)

0 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000204036.jpg seconds:  18.185779571533203
1 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000082052.jpg seconds:  34.692858934402466
2 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000083766.jpg seconds:  58.84759855270386
3 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000404988.jpg seconds:  77.67152690887451
4 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000071281.jpg seconds:  96.31073999404907
5 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000288002.jpg seconds:  112.83107781410217
6 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000535668.jpg seconds:  132.89531207084656
7 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000058915.jpg seconds:  152.9195475578308
8 Image Done: /content/drive/MyDrive/CS685/val2014/COCO_val2014_000000062549.jpg seconds:  171.62828636169434
9 Image Done: 

In [ ]:
!ls

demo.ipynb	     __pycache__	     utils.py
extracting_data.py   README.md		     visualizing_image.py
modeling_frcnn.py    requirements.txt
processing_image.py  stfeatures_149-415,pkl


In [ ]:
with open("/content/drive/MyDrive/ICDl/stfeatures_test_0-219.pkl", 'wb') as f:
  pickle.dump(s_test_features, f)
len(s_test_features)

219

In [ ]:
with open("/content/drive/MyDrive/ICDl/odtl_test_0-219.pkl", 'wb') as f:
  pickle.dump(od_test_list, f)